In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_squared_error
from sklearn.model_selection import train_test_split
np.random.seed(42) # set seed to keep results reproducable

In [9]:
# download all of the datasets into a list of dataframes
group_names = ['groupA', 'groupB', 'groupC']
datasets = []
for name in group_names:
    datasets.append(pd.read_csv(f"./Project2_Data/{name}.txt", header=None))
    # normalize each column 0 and 1
    for i in range(2):
        datasets[-1][i] = datasets[-1][i]/max(datasets[-1][i])
# convert all datasets to numpy matrixes
datasets = [dataset.to_numpy() for dataset in datasets]

 The unipolar activiation function: $\frac{1}{1 + e^{-k*net}}$

In [168]:
class Perceptron:
    def __init__(self, k: float, a: float):
        """
        Initalizes a new perceptron.

        Args:
            k: The gain for the activation function.
            a: The learning rate
        """
        self.weights = np.random.rand(3) - 0.5
        self.k = k
        self.a = a

    def _activation_func(self, x) -> float:
        """Calculated the activation function on input."""
        return 1/(1 + np.exp(-self.k * x))

    def fit(self, x: np.array, true_y: np.array, error_thres: float, max_epochs: int=5000):
        """
        Fits the perceptron to the input dataset.

        Args:
            x: The input test data
            true_y: The actual classification for the input classes
            error_thres: The amount of error that the model should stop training at
            max_epochs: The max number of times the model should be trained on the dataset
        """
        # add a column of 1's to make calculations simpler
        extra_col = np.ones(x.shape[0], dtype=x.dtype)
        x = np.column_stack((x, extra_col))
        print(mean_squared_error(true_y, self._activation_func(x @ self.weights)))

        # training the model
        for _ in range(max_epochs):
            for pattern, true_val in zip(x, true_y):
                prediction = self._activation_func(np.dot(pattern, self.weights))
                # find change in weights
                delta_w = self.a * (true_val - prediction) * pattern
                # update weights
                self.weights += delta_w
            # calculate the mean squared error to see if more training is needed
            if mean_squared_error(true_y, self._activation_func(x @ self.weights)) < error_thres:
                break
        print(mean_squared_error(true_y, self._activation_func(x @ self.weights)))

    
    def eval(self, x: np.array) -> np.array:
        """Calculates the model's prediction on x."""
        extra_col = np.ones(x.shape[0], dtype=x.dtype)
        x = np.column_stack((x, extra_col))
        return self._activation_func(x @ self.weights)

An example of using this class on group A dataset

In [169]:
percept_test = Perceptron(k=30, a=0.001)
X = datasets[0][:,:2]
true_y = datasets[0][:,2]

In [171]:
percept_test.fit(X, true_y, error_thres=0.00001, max_epochs=5000)

0.49951558148112923
9.998432886937813e-06


In [173]:
mean_squared_error(true_y, percept_test.eval(X))

9.998432886937813e-06